In [25]:
library(Seurat)
library(SingleCellExperiment)
library(MuSiC)
library(MuSiC2)
library(dplyr)
library(zellkonverter)

library(reshape2)
library(reshape)

In [2]:
#Bulk RNA-seq data
exprsFile <- 'bulk/expression_data_names.tsv'
exprs <- read.table(exprsFile, header=TRUE, sep="\t",row.names=1,as.is=TRUE)
exprs <- as.data.frame(exprs)
exprs <- exprs %>% 
  select(-c("X72"))
exprs <- as.matrix(exprs)
minimalSet <- ExpressionSet(assayData=exprs)

In [5]:
#SC RNA-seq data

library(Seurat)
library(SeuratData)
library(SeuratDisk)

Convert("scRNA/adata_raw_wctrl.h5ad", dest = "h5seurat", overwrite = TRUE)


Warning message:
"Unknown file type: h5ad"
Warning message:
"'assay' not set, setting to 'RNA'"
Creating h5Seurat file for version 3.1.5.9900

Adding X as data

Adding X as counts

Adding meta.features from var



In [6]:
sce <- LoadH5Seurat("scRNA/adata_raw_wctrl.h5seurat",meta.data = FALSE, misc = FALSE)


Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding tool-specific results



In [7]:
meta <- read.table("scRNA/adata_clustered_wctrl_harm_cluster_merged_meta.tsv",sep="\t",row.names=1,header=T,as.is=TRUE)

sce <- AddMetaData(object = sce,metadata = meta)
Idents(sce) <- "leiden_0.25"
table(Idents(sce))


   6    8    2    0    1    5    7    4    9    3 
1436  668 2426 3839 2586 1614  873 1814  379 2363 

In [8]:
markers <- FindAllMarkers(sce,test.use = 'wilcox',logfc.threshold = 2,min.pct = 0.25)
head(markers)

Calculating cluster 6

Calculating cluster 8

Calculating cluster 2

Calculating cluster 0

Calculating cluster 1

Warning message in FindMarkers.default(object = data.use, slot = data.slot, counts = counts, :
"No features pass logfc.threshold threshold; returning empty data.frame"
Calculating cluster 5

Calculating cluster 7

Calculating cluster 4

Calculating cluster 9

Calculating cluster 3



,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
MLLT3,0,3.063258,0.974,0.489,0,6,MLLT3
SLIT2,0,2.856964,0.841,0.299,0,6,SLIT2
NRXN3,0,2.174819,0.953,0.700,0,6,NRXN3
RNF220,0,2.141785,0.847,0.519,0,6,RNF220
FREM1,0,2.126039,0.809,0.274,0,6,FREM1
XACT,0,-2.747762,0.334,0.804,0,6,XACT


In [12]:
sce_obj <- as.SingleCellExperiment(sce)

In [150]:
#prepare data, only take overlapping genes between bulk and single-cell experiment

In [10]:
bulk.mtx <- exprs(minimalSet)

In [11]:
marker_genes <- unique(markers$gene)
marker_genes <- intersect(marker_genes,row.names(bulk.mtx))
print(length(marker_genes))

[1] 222


In [13]:
overlapping_genes <- intersect(row.names(sce_obj),row.names(bulk.mtx))
print(length(overlapping_genes))

[1] 23443


In [14]:
sce_obj <- sce_obj[overlapping_genes,]

In [15]:
bulk.mtx <- bulk.mtx[overlapping_genes,]
bulk.mtx <- na.omit(bulk.mtx)
dim(bulk.mtx)

[1] 23443    95

In [16]:
table(sce_obj$sample)


       iPSC_ctrl iPSC_prepat_d5_1 iPSC_prepat_d5_2 
            2366             8068             7564 

In [17]:
# Estimate cell type proportions
Est.prop.GSE50244 = music_prop(bulk.mtx = bulk.mtx,
                               sc.sce = sce_obj, 
                               clusters = 'leiden_0.25',
                               samples = 'sample',
                               select.ct = NULL
                               , verbose = T)

Creating Relative Abudance Matrix...

Creating Variance Matrix...

Creating Library Size Matrix...

Used 20331 common genes...

Used 9 cell types in deconvolution...

X1 has common genes 14097 ...

X10 has common genes 14396 ...

X11 has common genes 14590 ...

X12 has common genes 14685 ...

X13 has common genes 14832 ...

X14 has common genes 14620 ...

X15 has common genes 14600 ...

X16 has common genes 14697 ...

X17 has common genes 14813 ...

X18 has common genes 14677 ...

X19 has common genes 14945 ...

X2 has common genes 14261 ...

X20 has common genes 15097 ...

X21 has common genes 14597 ...

X22 has common genes 14676 ...

X23 has common genes 15049 ...

X24 has common genes 15091 ...

X25 has common genes 13916 ...

X26 has common genes 14514 ...

X27 has common genes 14477 ...

X28 has common genes 14660 ...

X29 has common genes 14930 ...

X3 has common genes 14331 ...

X30 has common genes 14466 ...

X31 has common genes 14727 ...

X32 has common genes 14495 ...

X33 

In [18]:
names(Est.prop.GSE50244)

m.prop.GSE50244 = rbind(melt(Est.prop.GSE50244$Est.prop.weighted), 
                        melt(Est.prop.GSE50244$Est.prop.allgene))

colnames(m.prop.GSE50244) = c('Sub', 'CellType', 'Prop')
m.prop.GSE50244$CellType = factor(m.prop.GSE50244$CellType, levels = unique(m.prop.GSE50244$CellType))
m.prop.GSE50244$Method = factor(rep(c('MuSiC', 'NNLS'), each = 95*length(unique(m.prop.GSE50244$CellType))), levels = c('MuSiC', 'NNLS'))
write.table(m.prop.GSE50244,"bulk/music_wctrl/proportions_0.25.tsv",sep="\t")

[1] "Est.prop.weighted" "Est.prop.allgene"  "Weight.gene"      
[4] "r.squared.full"    "Var.prop"

Warning message in type.convert.default(X[[i]], ...):
"'as.is' should be specified by the caller; using TRUE"
Warning message in type.convert.default(X[[i]], ...):
"'as.is' should be specified by the caller; using TRUE"
Warning message in type.convert.default(X[[i]], ...):
"'as.is' should be specified by the caller; using TRUE"
Warning message in type.convert.default(X[[i]], ...):
"'as.is' should be specified by the caller; using TRUE"
